In [ ]:
from google.colab import drive
import numpy as np
import scipy.io as sc
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.applications.efficientnet import preprocess_input
from keras.applications import EfficientNetB0
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam


IMAGE_SIZE = (224,224)
BATCH_SIZE = 64
NUM_OF_CLASSES = 13
AUTOTUNE = tf.data.AUTOTUNE
ROOT_DIR = 'Paintings91/'

In [ ]:
drive.mount('/content/drive')
!unzip drive/MyDrive/Paintings91.zip

In [ ]:
def preprocess_image(filename, label):

    image = tf.io.read_file(ROOT_DIR + 'Images/' + filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = preprocess_input(image)

    return image, label

In [ ]:
def get_data_sets():

  dir_suffix = ""
  file_suffix = ""

  if(NUM_OF_CLASSES == 13):
    dir_suffix = "_Style"
    file_suffix = "_style"

  labels = sc.loadmat(file_name = ROOT_DIR +"/Labels" + dir_suffix + "/labels"+ file_suffix + ".mat", mdict=None, appendmat=True)
  testset = sc.loadmat(file_name = ROOT_DIR +"/Labels" + dir_suffix + "/testset" + file_suffix + ".mat", mdict=None, appendmat=True)
  trainset = sc.loadmat(file_name = ROOT_DIR +"/Labels" + dir_suffix + "/trainset" + file_suffix + ".mat", mdict=None, appendmat=True)
  image_names = sc.loadmat(file_name = ROOT_DIR +"/Labels" + dir_suffix +"/image_names" + file_suffix + ".mat", mdict=None, appendmat=True)

  image_names['image_names' + file_suffix] = np.array(
    list(map(lambda name: name[0][0].replace('Ã', 'Р'),
             image_names['image_names' + file_suffix ])))

  train_indices = np.where(trainset['trainset' + file_suffix].ravel())
  test_indices = np.where(testset['testset' + file_suffix].ravel())

  X_train = image_names['image_names' + file_suffix][train_indices]
  y_train = labels['labels' + file_suffix][train_indices]

  X_test = image_names['image_names' + file_suffix][test_indices]
  y_test = labels['labels' + file_suffix][test_indices]

  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify = y_train)

  train_ds =  tf.data.Dataset.from_tensor_slices((X_train,y_train)).map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
  val_ds =  tf.data.Dataset.from_tensor_slices((X_val,y_val)).map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
  test_ds =  tf.data.Dataset.from_tensor_slices((X_test,y_test)).map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

  return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_data_sets()

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False)
x = base_model.output
for layer in base_model.layers[:213]:
  layer.trainable = False
x = GlobalAveragePooling2D()(x)
output = Dense(NUM_OF_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer = Adam(1e-3), loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 8, restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience = 5, min_lr = 1e-4)

In [ ]:
model.fit(train_ds, validation_data=(val_ds), callbacks=[early_stopping, reduce_lr], epochs = 50)